In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
import warnings
import seaborn as sns
from tensorflow.keras.metrics import *

warnings.filterwarnings('ignore')

print(tf.__version__)
print(np.__version__)
print(pd.__version__)

In [ ]:
data = pd.read_csv('Diabetes Classification.csv').sample(frac=1) #sample is used to gererate the random data otherwise first 5 datas is considered
data.head()
data.describe()
#data.isna().sum() #the no of missing data in each column


In [ ]:
#data_preprocessing
features = data[['Diagnosis','Chol','TG','HDL','LDL', 'Cr','BUN']]

# features['Age']=features['Age'].fillna(42)
# age2=pd.get_dummies(features['age'])
# features=features.drop(columns=['Age','BUN']
# features=pd.concat(['features','age2'],axis=1)

#to train the data use the command
target=data['BMI']

train_size=int(0.8*len(data))
X_train, X_test = features.values[ :train_size, :],features.values[train_size: , :]
y_train, y_test = target.values[:train_size],target.values[train_size:]

print(len(X_train))


In [ ]:
sns.pairplot(features)

In [ ]:
corr = features.corr()
cmap = sns.diverging_palette(250 , 10 , as_cmap=True)
plt.figure(figsize=(8,8))
sns.heatmap(corr, square=True, cmap=cmap, annot=True)

In [ ]:
class LinearModel:
    def __init__(self):
        # y_pred= W x + b (y=Mx+c)
        self.initializer = tf.keras.initializers.GlorotUniform()
        
        #self.W= tf.Variable(13.0)
        #self.b = tf.Variable(4.0)
    
    def loss(self, y, y_pred): 
        return tf.reduce_mean(tf.abs(y - y_pred))
        
    def train(self, X, y, lr=0.00001,epochs=20, verbose=True):
        X = np.asarray(X, dtype=np.float32)       
        y = np.asarray(y, dtype=np.float32).reshape((-1,1))  # [1,2,3] = [[1],[2],[3]]
        
        self.W = tf.Variable(
            initial_value=self.initializer(shape=(len(X[0]),1),dtype='float32'))
        self.b = tf.Variable(
            initial_value=self.initializer(shape=(1,),dtype='float32'))

        def train_step():
            with tf.GradientTape() as t:
                current_loss=self.loss(y,self.predict(X))
            
            dw, db = t.gradient(current_loss, [self.W, self.b])
            self.W.assign_sub(lr * dw) # W -= Lr * dw
            self.b.assign_sub(lr * db)
            return current_loss
                
        for epoch in range(epochs):
                current_loss = train_step()
                if verbose:
                    print(f'Epoch {epoch}: Loss: {current_loss.numpy()}')
                    
    def predict(self, X):
        return tf.matmul(X,self.W)+self.b


In [ ]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu'), #activation functions are non-linear operations that act on tensors , Relu - rectified linear unit
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1) #for regression model
])    
    

In [ ]:
model = LinearModel()
model.train(X_train,y_train, epochs =130)

In [ ]:
#equal to def train() function in linear model

sgd_optimizer = tf.keras.optimizers.SGD()  #Stochastic gradient descent (SGD) optimiser
loss_fn= tf.keras.losses.MAE
model.compile(
    optimizer=sgd_optimizer,
    loss=loss_fn,
    metrics=[tf.keras.metrics.MAE]
)
    

In [ ]:
#equals to def train() function in linear model

adam_optimizer = tf.keras.optimizers.Adam()  #Adaptive Moment Estimation(Adam) optimiser is used to reduce the loss function
loss_fn= tf.keras.losses.MAE
model.compile(
    optimizer=adam_optimizer,
    loss=loss_fn,
    metrics=[tf.keras.metrics.MAE]
)

#compare the result with the sgd optimiser, this one is providing better results
    

In [ ]:
#adam optimizer with few other metrics

def R_squared(y_true,y_pred):
    residual=tf.reduce_sum(tf.sum(y_true-y_pred))
    total=tf.reduce_sum(tf.sum(y_true-tf.reduce_mean(y_true)))
    r2 = 1 - residual / total
    return r2

adam_optimizer = tf.keras.optimizers.Adam()  #Adaptive Moment Estimation(Adam) optimiser is used to reduce the loss function
loss_fn= tf.keras.losses.MAE
model.compile(
    optimizer=adam_optimizer,
    loss=loss_fn,
    metrics=[
        tf.keras.metrics.MAE,
        tf.keras.metrics.MSE,
        R_squared
    ]
)


In [ ]:
model.fit(X_train, y_train, epochs=10)

In [ ]:
model.evaluate(X_test, y_test)